In [ ]:
#Repulsion energy 
with open("ion_ion.dat","r") as repulsion:
    repulsion_energy = float(repulsion.readline().strip())

#print("Ion-ion Electrostatic Energy :", repulsion_energy)

In [ ]:
import numpy as np

# Open the file and read all lines
with open("one_electron_integrals.dat", "r") as oei:
    lines = oei.readlines()  # Read the entire file into a list

# Extract the first line and parse orbital numbers
first_line = lines[0].strip().split()
noccp = int(first_line[0])  # Number of occupied spatial orbitals
nvirt = int(first_line[1])  # Number of virtual orbitals

# Total spatial orbitals
norb = noccp + nvirt

# Construct spatial one-electron integral matrix
one_electron_tensor = np.zeros((norb, norb))

# Process the remaining lines (integral data)
for line in lines[1:]:  # Skip the first line
    data = line.strip().split()
    if len(data) == 3:
        i, j, value = int(data[0])-1, int(data[1])-1, float(data[2])
        one_electron_tensor[i, j] = value
        one_electron_tensor[j, i] = value  # Ensure H(i,j) = H(j,i)

# Construct the spin-orbital matrix 
num_spin_orb = 2 * norb
one_electron_spin_tensor = np.zeros((num_spin_orb, num_spin_orb))

# Fill in the spin-orbital tensor
for p in range(norb):
    for q in range(norb):
        value = one_electron_tensor[p, q]
        
        # alpha-alpha block
        one_electron_spin_tensor[2*p, 2*q] = value
        # beta-beta block
        one_electron_spin_tensor[2*p + 1, 2*q + 1] = value

# Print number of Occupied and Virtual Orbitals
print(f"Number of occupied spatial orbitals: {noccp}")
print(f"Number of virtual spatial orbitals: {nvirt}")

# Print One-electron integral tensor 
print("One-electron integral tensor:")
print(one_electron_tensor)

# Print spin one-electron integral tensor
print("\nSpin-Orbital One-Electron Integral Tensor (Block Diagonal):")
print(one_electron_spin_tensor)


In [ ]:
import numpy as np

# Function to read and symmetrize two-electron integrals
def construct_tei(filename):
    with open(filename, "r") as tei:
        # Read first line (number of occupied and virtual orbitals, number of integrals)
        noccp, nvirt, value2 = map(int, tei.readline().split())
        norb = noccp + nvirt  # Total number of spatial orbitals
        tensor = np.zeros((norb, norb, norb, norb))

        # Read tensors
        for _ in range(value2):
            p, q, r, s, val = map(float, tei.readline().split())
            p, q, r, s = int(p) - 1, int(q) - 1, int(r) - 1, int(s) - 1  # Adjust zero indexing

            # Store values in all equivalent positions
            for a, b, c, d in [
                (p, q, r, s), (p, q, s, r), (q, p, r, s), (q, p, s, r),
                (r, s, p, q), (r, s, q, p), (s, r, p, q), (s, r, q, p)
            ]:
                tensor[a, b, c, d] = val
    return tensor, norb


# Process and transform Coulomb tensor
coulomb_tensor, norbs = construct_tei("two_electron_integrals_coulomb.dat")

# Process and transform Exchange tensor
exchange_tensor, _ = construct_tei("two_electron_integrals_exchange.dat")

# Initialize spin-orbital two-electron integral tensor
num_spin_orb = 2 * norbs
spin_coulomb_tensor = np.zeros((num_spin_orb, num_spin_orb, num_spin_orb, num_spin_orb))
spin_exchange_tensor = np.zeros((num_spin_orb, num_spin_orb, num_spin_orb, num_spin_orb))

# Transform to spin-orbital 
for p in range(norbs):
    for q in range(norbs):
        for r in range(norbs):
            for s in range(norbs):
                value_coulomb = coulomb_tensor[p, q, r, s]
                value_exchange = exchange_tensor[p, q, r, s]

                # Same spin contributions
                spin_coulomb_tensor[2*p, 2*q, 2*r, 2*s] = value_coulomb
                spin_coulomb_tensor[2*p+1, 2*q+1, 2*r+1, 2*s+1] = value_coulomb

                spin_exchange_tensor[2*p, 2*q, 2*r, 2*s] = value_exchange
                spin_exchange_tensor[2*p+1, 2*q+1, 2*r+1, 2*s+1] = value_exchange

                # Mixed spin contributions
                spin_coulomb_tensor[2*p, 2*q, 2*r+1, 2*s+1] = value_coulomb
                spin_coulomb_tensor[2*p+1, 2*q+1, 2*r, 2*s] = value_coulomb

                spin_exchange_tensor[2*p, 2*q, 2*r+1, 2*s+1] = value_exchange
                spin_exchange_tensor[2*p+1, 2*q+1, 2*r, 2*s] = value_exchange

# Print results
print("Coulomb Tensor Shape (Spatial):", coulomb_tensor.shape)
print("Exchange Tensor Shape (Spatial):", exchange_tensor.shape)

print("\nCoulomb Tensor Shape (Spin-Orbital):", spin_coulomb_tensor.shape)
print("Exchange Tensor Shape (Spin-Orbital):", spin_exchange_tensor.shape)
